<a href="https://colab.research.google.com/github/Tihara-Jay/FYP-model-implementation/blob/main/Model_Conversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from torch import nn
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
class BERTClassifier(nn.Module):
    def __init__(self, bert_model_name, num_classes):
        super(BERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)

        self.dropout = nn.Dropout(0.1)
        self.relu =  nn.ReLU()
        self.fc1 = nn.Linear(768,512)
        self.fc2 = nn.Linear(512,1)


    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output

        x = self.fc1(pooled_output)
        x = self.relu(x)
        x = self.dropout(x)

        logits = self.fc2(x).squeeze(-1)
        return logits

In [4]:
!pip install coremltools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 8.5 MB/s eta 0:00:00


In [8]:
import coremltools as ct
import torch

bert_model_name = 'bert-base-uncased'
num_classes = 2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = BERTClassifier(bert_model_name, num_classes).to(device)
#load the saved stAte params of the pre-trained model
model.load_state_dict(torch.load("/content/drive/MyDrive/FYP/Implementation/FYP_Implementation/Spam_image_classification.pth", map_location=torch.device('cpu')))


dummy_input_ids = torch.zeros((1, 128), dtype=torch.long)
dummy_attention_mask = torch.zeros((1, 128), dtype=torch.long)
traced_model = torch.jit.trace(model, (dummy_input_ids, dummy_attention_mask))
input_shape = (1, 128)


# Convert the model to a core ml model
coreml_model = ct.convert(
    traced_model,
    source='pytorch',
    convert_to="neuralnetwork",
    inputs=[ct.TensorType(name='input_ids', shape=input_shape),
            ct.TensorType(name='attention_mask', shape=input_shape)],
    outputs=['output']
)

coreml_model.save("/content/drive/MyDrive/FYP/Implementation/FYP_Implementation/SpamImageClassification.mlmodel")

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4193: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/jit/_trace.py:1102: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1 / 1 (100.0%)
Greatest absolute difference: 0.16414809226989746 at index (0,) (up to 1e-05 allowed)
Greatest relative difference: 0.15194438973092536 at index (0,) (up to 1e-05 allowed)
  _check_trace(
Translating MIL ==> NeuralNetwork Ops: 100%|██████████| 713/713 [00:19<00:00, 36.36 ops/s]
